# Домашнее задание к занятию «Классификация: Логистическая регрессия и SVM»

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

* В домашнем задании нужно решить задачу классификации физических лиц по уровню дохода. Данные для обучения модели хранятся в файле adult.csv
* Целевая переменная – уровень дохода income, который принимает два значения <=50K и >50K, поэтому классификация бинарная. Остальные признаки описывают персональную информацию – возраст, образование, семейное положение и т. д. 
* Задачу классификации нужно решить при помощи модели логистической регрессии и модели опорных векторов.

In [ ]:
data = pd.read_csv('adult.csv', na_values='?')
data.head()

Проведите первичный анализ:
* Проверьте данные на пропуски. Удалите в случае обнаружения. Предложите альтернативный способ работы с пропусками
* Постройте 1-2 графика на выбор. Визуализация должна быть основана на исследуемых данных и быть полезной (из графика можно сделать вывод об особенностях датасета/класса/признака)
* Преобразуйте категориальные признаки

In [ ]:
data.info()

In [ ]:
# Удаляем пропуски по столбцам - workclass, occupation, native-country 
# На основании имеющихся данных по заполненным столбцам сложно заполнить корректно категориальные данные
data = data.dropna()

In [ ]:
# Проверим дубликаты
duplicates = data.duplicated()
num_duplicates = duplicates.sum()
num_duplicates

In [ ]:
# Удалим дубликаты из данных для анализа
data = data.drop_duplicates()
data.info()

In [ ]:
# Предположим зависимость показателей возраст - доход, проверим это визуально
sns.histplot(data=data, x='age', hue='income', bins=30, stat='density', common_norm=False)
plt.title('Распределение возраста по классам дохода')
plt.xlabel('Возраст')
plt.ylabel('Плотность')
plt.show()

In [ ]:
# Определим взаимосвязь уровня образования с показателем дохода (рассмотрим выборку людей с доходом свыше 50к)
data_group = data[(data['income'] == '>50K')]
data_group = data_group[['education', 'income']].groupby('education').count().sort_values(by='income')
data_group.plot(kind='barh', color='skyblue')
plt.xlabel('Количество людей с доходом >50K')
plt.title('Уровень образования людей с доходом >50K')
plt.show()

In [ ]:
# Преобразуем категориальные признаки
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()
print('Категориальные признаки:', categorical_cols)

In [ ]:
select_сolumns = data[['age', 'education', 'income']]
select_сolumns

In [ ]:
X = pd.get_dummies(select_сolumns, columns=['education'])
del X['income']
X.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
le.fit(data['income'])

In [ ]:
le.classes_

In [ ]:
y = pd.Series(data=le.transform(data['income']))
y.head()

Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Обучение модели методом Логистической регрессии

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model.fit(X_train, y_train)
predictions = model.predict(X_test)

Для тестового множества предскажите уровень дохода 

In [ ]:
predictions

In [ ]:
model.predict_proba(X_test)

Посчитайте точность предсказания моделей. Для этого используйте встроенную функцию score.

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
model.score(X_train, y_train) 

In [ ]:
log_acc = model.score(X_test,y_test)
log_acc

Обучение модели методом опорных векторов на обучающем множестве.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
# SVM с RBF ядром (по умолчанию)
svm_rbf = SVC()
svm_rbf.fit(X_train, y_train)

In [ ]:
svm_rbf.score(X_train, y_train) 

In [ ]:
svm_rbf_acc = svm_rbf.score(X_test,y_test)
svm_rbf_acc

In [ ]:
# SVM с RBF ядром и StandardScaler - масштабирование признаков
svm_rbf_scaler = make_pipeline(StandardScaler(), SVC()) 
svm_rbf_scaler.fit(X_train, y_train)

In [ ]:
svm_rbf_scaler.score(X_train, y_train) 

In [ ]:
svm_rbf_scaler_acc = svm_rbf_scaler.score(X_test,y_test)
svm_rbf_scaler_acc

In [ ]:
# SVM с линейным ядром (linear)
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train, y_train)

In [ ]:
svm_linear.score(X_train, y_train) 

In [ ]:
svm_linear_acc = svm_linear.score(X_test,y_test)
svm_linear_acc

Сформулируйте выводы по проделанной работе:
a) кратко опишите какие преобразования были сделаны с данными.
b) cравните точность двух моделей.
c) напишите свое мнение, в полной ли мере модели справились с поставленной задачей.
* Что по вашему мнению нужно сделать, чтобы улучшить результат?

In [ ]:
print(f'Результаты: \nLogistic Regression: {log_acc:.3f},\n'
    f'SVM RBF: {svm_rbf_acc:.3f}, \n'
    f'SVM RBF StandardScaler: {svm_rbf_scaler_acc:.3f}, \n'
    f'SVM Linear: {svm_linear_acc:.3f}')

* В ДЗ проанализирован датасет adult.csv
* Мы предсказываем целевую переменную income на основании двух признаков: age, education.
* Все рассмотренные модели показали хорошее качество классификации.
* Масштабирование признаков StandardScaler значимо влияет на работу SVM
* Наилучший результат на тестовой выборке получен при применении SVM RBF с использованием StandardScaler